In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("train_BRCpofr.csv")

In [3]:
df_n = pd.get_dummies(df)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89392 entries, 0 to 89391
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              89392 non-null  int64 
 1   gender          89392 non-null  object
 2   area            89392 non-null  object
 3   qualification   89392 non-null  object
 4   income          89392 non-null  object
 5   marital_status  89392 non-null  int64 
 6   vintage         89392 non-null  int64 
 7   claim_amount    89392 non-null  int64 
 8   num_policies    89392 non-null  object
 9   policy          89392 non-null  object
 10  type_of_policy  89392 non-null  object
 11  cltv            89392 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 8.2+ MB


In [5]:
df_n

id  marital_status  vintage  claim_amount    cltv  gender_Female  \
0          1               1        5          5790   64308              0   
1          2               0        8          5080  515400              0   
2          3               1        8          2599   64212              0   
3          4               0        7             0   97920              1   
4          5               1        6          3508   59736              0   
...      ...             ...      ...           ...     ...            ...   
89387  89388               0        6             0   95736              0   
89388  89389               1        0          3897   67596              0   
89389  89390               0        4          3363  114120              1   
89390  89391               0        8             0   71772              1   
89391  89392               0        6             0  108144              1   

       gender_Male  area_Rural  area_Urban  qualification_Bachelor  ...  \
0                1           0           1                       1  ...   
1                1           1           0                       0  ...   
2                1           0           1                       1  ...   
3                0           1           0                       0  ...   
4                1           0           1                       0  ...   
...            ...         ...         ...                     ...  ...   
89387            1           1           0                       0  ...   
89388            1           0           1                       1  ...   
89389            0           0           1                       1  ...   
89390            0           1           0                       0  ...   
89391            0           1           0                       0  ...   

       income_<=2L  income_More than 10L  num_policies_1  \
0                0                     0               0   
1                0                     0               0   
2                0                     0               0   
3                0                     0               0   
4                0                     1               0   
...            ...                   ...             ...   
89387            0                     1               0   
89388            0                     0               0   
89389            0                     0               0   
89390            0                     0               0   
89391            0                     0               0   

       num_policies_More than 1  policy_A  policy_B  policy_C  \
0                             1         1         0         0   
1                             1         1         0         0   
2                             1         1         0         0   
3                             1         1         0         0   
4                             1         1         0         0   
...                         ...       ...       ...       ...   
89387                         1         0         1         0   
89388                         1         1         0         0   
89389                         1         0         1         0   
89390                         1         0         1         0   
89391                         1         0         1         0   

       type_of_policy_Gold  type_of_policy_Platinum  type_of_policy_Silver  
0                        0                        1                      0  
1                        0                        1                      0  
2                        0                        1                      0  
3                        0                        1                      0  
4                        1                        0                      0  
...                    ...                      ...                    ...  
89387                    0                        1                      0  
89388                    0                        1   

In [6]:
outliers = []
def detect_outliers_iqr(data):
    data = sorted(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    # print(q1, q3)
    IQR = q3-q1
    lwr_bound = q1-(1.5*IQR)
    upr_bound = q3+(1.5*IQR)
    # print(lwr_bound, upr_bound)
    for i in data: 
        if (i<lwr_bound or i>upr_bound):
            outliers.append(i)
    return outliers# Driver code
sample_outliers = detect_outliers_iqr(df_n['claim_amount'])
print("Outliers from IQR method: ", len(sample_outliers))

Outliers from IQR method:  2258


In [7]:
claim_Amount = df_n['claim_amount']

In [8]:
# Computing 10th, 90th percentiles and replacing the outliers
tenth_percentile = np.percentile(df_n['claim_amount'], 10)
ninetieth_percentile = np.percentile(df_n['claim_amount'], 90)
# print(tenth_percentile, ninetieth_percentile)
b = np.where(df_n['claim_amount']<tenth_percentile, tenth_percentile, df_n['claim_amount'])
b = np.where(b>ninetieth_percentile, ninetieth_percentile, b)
# print("Sample:", sample)
print("New array:",b)

New array: [5790. 5080. 2599. ... 3363.    0.    0.]


In [9]:
df_n.columns

Index(['id', 'marital_status', 'vintage', 'claim_amount', 'cltv',
       'gender_Female', 'gender_Male', 'area_Rural', 'area_Urban',
       'qualification_Bachelor', 'qualification_High School',
       'qualification_Others', 'income_2L-5L', 'income_5L-10L', 'income_<=2L',
       'income_More than 10L', 'num_policies_1', 'num_policies_More than 1',
       'policy_A', 'policy_B', 'policy_C', 'type_of_policy_Gold',
       'type_of_policy_Platinum', 'type_of_policy_Silver'],
      dtype='object')

In [10]:
df_n['claim_amount'] = b

In [11]:
df_n.to_csv('input.csv')

In [12]:
X=df_n.drop(['cltv'], axis=1)
y=df_n['cltv']

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 43)

In [14]:
from sklearn.ensemble import GradientBoostingRegressor

In [40]:
from sklearn.model_selection import GridSearchCV
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=10000,min_samples_leaf=300,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10), 
param_grid = param_test1, scoring='r2',n_jobs=4, cv=10)
gsearch1.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=GradientBoostingRegressor(max_depth=8,
                                                 max_features='sqrt',
                                                 min_samples_leaf=300,
                                                 min_samples_split=10000,
                                                 random_state=10,
                                                 subsample=0.8),
             n_jobs=4, param_grid={'n_estimators': range(20, 81, 10)},
             scoring='r2')

In [41]:
from sklearn.metrics import r2_score
y_pred = gsearch1.predict(X_test)
score = r2_score(y_test,y_pred)
print('r2 socre is ',score)

r2 socre is  0.15899621988905543


In [43]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xg
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [49]:
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
#dtc =  Ridge()
#dtc = LinearRegression()

gbm =  GradientBoostingRegressor()
xgb = XGBRegressor()
rf = RandomForestRegressor(max_depth=10, min_samples_leaf=2, min_samples_split=0.01,n_estimators=10000)
lgb = LGBMRegressor()
cat = CatBoostRegressor()

#clf = [('dtc',dtc),('rfc',rfc),('knn',knn),('DT',DT),('xgb',xgb)] #list of (str, estimator)
clf = [('rf',rf),('xgb',xgb),('lgb',lgb),('cat',cat)]

In [ ]:
stack_model = StackingRegressor(estimators=clf, final_estimator=gbm)
#score = cross_val_score(stack_model,X_train,y_train,cv = 5,scoring = 'accuracy')
stack_model.fit(X_train,y_train)
y_pred = stack_model.predict(X_test)
print('-'*30)
r2 = r2_score(y_test,y_pred)
print('R2: ', r2)